# 01. Machine Learning
 - Vamos estabelecer primariamente um modelo baseline para melhor interpretabilidade.
 - Nosso objetivo será controlar o ERRO TIPO 2(Falso Negativo) para não perder receita recorrente (LTV - Lifetime Value)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregando os dados processados
# Como salvamos sem index, não precisamos nos preocupar com colunas extras
X_train = pd.read_csv('../data/processed/X_train_processed.csv')
y_train = pd.read_csv('../data/processed/y_train_processed.csv')

X_test = pd.read_csv('../data/processed/X_test_processed.csv')
y_test = pd.read_csv('../data/processed/y_test_processed.csv')

# O y é carregado como DataFrame, mas modelos preferem vetor (Series 1D)
# O .ravel() resolve isso achatando o array
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

print(f"Dados carregados! Treino: {X_train.shape}, Teste: {X_test.shape}")